<h2>Nhập data vào Mysql</h2>


<h5>Phần này sẽ tiến hành đưa data vào hệ quản trị cơ sở dữ liệu MySQL</h5>

In [1]:
import pandas as pd
import pymysql
from sqlalchemy import create_engine
from concurrent.futures import ThreadPoolExecutor

In [2]:
hostname = "localhost"
database = "football"
username = "root"
password = "123456"

engine = create_engine("mysql+pymysql://{user}:{pw}@{host}/{db}".format(host=hostname, 
                                                                        db=database, 
                                                                        user=username, 
                                                                        pw=password))

connection = pymysql.connect(
    host = hostname,
    user = username,
    password = password,
    database = database
)

In [3]:
dataframes_dict = {
    # "tablename":"path",
    "future_matches": "./future_matches.csv",
}

def process_dataframe(df_name, file_path):
    df = pd.read_csv(file_path)
    # If want to replace all data, use if_exists="replace"
    df.to_sql(df_name, engine, if_exists="replace", index=True,index_label="id")
    # If want to append data, use if_exists="append"
    # df.to_sql(df_name, engine, if_exists="append", index=True,index_label="id")
    

# Create a ThreadPoolExecutor with the desired number of threads
executor = ThreadPoolExecutor(max_workers=4)

# Submit each dataframe processing task to the executor
futures = [executor.submit(process_dataframe, df_name, file_path) for df_name, file_path 
           in dataframes_dict.items()]

# Wait for all tasks to complete
for future in futures:
    future.result()

connection.close()
engine.dispose()